<a href="https://colab.research.google.com/github/Sankytanky100/ML-DL-Google-Collab-Projects/blob/main/AI_Model_Monitoring_for_Healthcare_Applications_Using_Logging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AI Model Monitoring for Healthcare Applications Using Logging

Introduction
In healthcare, AI models are used for critical tasks like disease prediction, patient risk assessment, and diagnostics. Monitoring these models is essential to ensure their reliability, accuracy, and compliance with regulations.

Logging is a powerful tool that helps in:

Tracing model behavior over time.
Debugging issues by recording anomalies.
Auditing for compliance with healthcare regulations.


In [ ]:
# Import necessary libraries
import logging
import numpy as np
import pandas as pd
import sys
import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


Data Preparation

In [ ]:
# Simulate a healthcare dataset
np.random.seed(42)

# Features: Age, Blood Pressure, Cholesterol Levels, etc.
data_size = 1000
data = pd.DataFrame({
    'age': np.random.randint(20, 80, data_size),
    'blood_pressure': np.random.randint(80, 180, data_size),
    'cholesterol': np.random.randint(150, 300, data_size),
    'heart_rate': np.random.randint(60, 100, data_size),
    'blood_sugar': np.random.randint(70, 150, data_size),
})

# Target variable: Disease Presence (1 for disease, 0 for no disease)
data['disease'] = np.where(
    (data['age'] > 50) &
    (data['blood_pressure'] > 140) &
    (data['cholesterol'] > 240), 1, 0
)

# Split the data
X = data.drop('disease', axis=1)
y = data['disease']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


Model Building and Training

In [ ]:
# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       185
           1       1.00      1.00      1.00        15

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



Implementing Logging

In [ ]:
# Configure logging
logger = logging.getLogger('ModelMonitoringLogger')
logger.setLevel(logging.INFO)

# Create handlers
stream_handler = logging.StreamHandler(sys.stdout)
file_handler = logging.FileHandler('model_monitoring.log')

# Create formatters and add them to handlers
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
stream_handler.setFormatter(formatter)
file_handler.setFormatter(formatter)

# Add handlers to the logger
logger.addHandler(stream_handler)
logger.addHandler(file_handler)


Making Predictions and Logging Outputs

In [ ]:
# Function to log predictions
def log_predictions(model, X_input):
    probabilities = model.predict_proba(X_input)
    predictions = model.predict(X_input)
    for i in range(len(predictions)):
        confidence = max(probabilities[i])
        logger.info(f"Prediction: {predictions[i]}, Confidence: {confidence:.2f}")

        # Log unusual predictions (e.g., low confidence)
        if confidence < 0.6:
            logger.warning(f"Low confidence prediction detected: {predictions[i]} with confidence {confidence:.2f}")

# Simulate new patient data
new_patient_data = pd.DataFrame({
    'age': [65],
    'blood_pressure': [150],
    'cholesterol': [260],
    'heart_rate': [85],
    'blood_sugar': [130],
})

# Log the prediction
log_predictions(model, new_patient_data)


2024-11-24 16:16:25,809 - INFO - Prediction: 1, Confidence: 0.94


INFO:ModelMonitoringLogger:Prediction: 1, Confidence: 0.94


Model Retraining and Version Tracking

In [ ]:
# Function to retrain the model and log the event
def retrain_model(model, X_train, y_train, version):
    model.fit(X_train, y_train)
    logger.info(f"Model retrained. New version: {version}")

# Simulate retraining the model
new_version = '1.1.0'
retrain_model(model, X_train, y_train, new_version)


2024-11-24 16:16:26,020 - INFO - Model retrained. New version: 1.1.0


INFO:ModelMonitoringLogger:Model retrained. New version: 1.1.0


Maintaining Audit Logs

In [ ]:
# Example of an audit log entry
def audit_log(action, details):
    timestamp = datetime.datetime.now().isoformat()
    audit_entry = f"{timestamp} - ACTION: {action} - DETAILS: {details}"
    # Append to audit log file
    with open('audit.log', 'a') as f:
        f.write(audit_entry + '\n')
    # Also log the audit entry
    logger.info(f"AUDIT LOG - {audit_entry}")

# Log an audit event
audit_log("ModelRetrain", f"Model version updated to {new_version}")


2024-11-24 16:16:26,037 - INFO - AUDIT LOG - 2024-11-24T16:16:26.035872 - ACTION: ModelRetrain - DETAILS: Model version updated to 1.1.0


INFO:ModelMonitoringLogger:AUDIT LOG - 2024-11-24T16:16:26.035872 - ACTION: ModelRetrain - DETAILS: Model version updated to 1.1.0


In [ ]:
# Display the model monitoring log
with open('model_monitoring.log', 'r') as f:
    logs = f.read()
    print(logs)


2024-11-24 16:16:25,809 - INFO - Prediction: 1, Confidence: 0.94
2024-11-24 16:16:26,020 - INFO - Model retrained. New version: 1.1.0
2024-11-24 16:16:26,037 - INFO - AUDIT LOG - 2024-11-24T16:16:26.035872 - ACTION: ModelRetrain - DETAILS: Model version updated to 1.1.0



In [ ]:
# Display the audit log
with open('audit.log', 'r') as f:
    audit_logs = f.read()
    print(audit_logs)


2024-11-24T16:16:26.035872 - ACTION: ModelRetrain - DETAILS: Model version updated to 1.1.0

